In [1]:
import matplotlib
import pandas as pd
from matplotlib import pyplot as plt
from glob import glob

from extract.jobs import parse_mallob, get_baseline
from extract.threads import parse_threads

In [15]:
pair = parse_threads("benchmark/test/")

pd.DataFrame(pair[1]).sort_values("average_time_per_cube", ascending=False)

,job,rank,instance,times_started,run_time,wait_time,processed_cubes,solves,solved,failed_cubes,interruptions,average_time_per_cube
560,4,14,0,1,887.421,0.000,1,0,True,0,0,887.420800
692,22,4,24,1,749.560,0.000,1,0,True,0,0,749.550000
772,28,6,31,1,724.000,5.010,1,0,True,0,0,718.990000
325,21,1,28,1,312.650,0.000,1,0,True,0,0,312.650000
36,37,13,56,1,922.410,12.840,3,0,False,3,0,303.186667
...,...,...,...,...,...,...,...,...,...,...,...,...
480,5,8,6,1,999.734,0.000,1,0,False,0,1,-1.000000
481,5,8,7,1,999.724,0.230,1,0,False,0,1,-1.000000
482,5,8,5,1,999.734,0.326,1,0,False,0,1,-1.000000
483,5,8,4,1,999.734,0.133,1,0,False,0,1,-1.000000


In [16]:
ids = pd.read_csv('competition_scenario', delimiter=' ')
# ids.filename = ids.filename.astype(str)

results = pd.read_csv('competition_results', delimiter=' ')
# results.filename = results.filename.astype(str)

ground_truth_df = pd.merge(ids, results, on='filename')

ground_truth_df.head(271)

,ID,Arrival,Priority,filename,hash,result
0,1,0,1,b2005-p4-12x12c10h7-Ser4-0.cnf.xz,bb12ece2fc8c01870418908a3dd8341f,unknown
1,2,0,1,mulhs16.smt2.cnf.xz,e3554a7be21336aa38a5fd7c7a1c9261,unknown
2,3,0,1,newpol7-6.cnf.xz,ac6578650f8c5211fbaa1f86a46ccf6e,unsat
3,4,0,1,g2-T85.2.1.cnf.xz,976649ddfe27c60bacb508ecd3499f39,unsat
4,5,0,1,6g_5color_170_030_04.cnf.xz,29be431e9dee1550957c8483be4532e4,unknown
...,...,...,...,...,...,...
266,267,0,1,preimage_80r_493m_160h_seed_457.cnf.xz,2e29192bee7f468c3c6e1b7088829575,sat
267,268,0,1,combined-crypto1-wff-seed-110-wffvars-500-cryp...,940aeb30818a6b0656c9906a32bcd7bc,sat
268,269,0,1,LED_round_29-32_faultAt_29_fault_injections_5_...,e47010231e43a6d68519a57c386a5509,unknown
269,270,0,1,abw-O-impcol_d.mtx-w101.cnf.xz,552b8976b37b0d8b58bc8edad91c5695,sat


In [ ]:
pd.DataFrame(parse_mallob("final/baseline2/"))

In [ ]:
ground_truth_result = ground_truth_df.result.rename("ground_truth_result")

dynamic_baseline_df = pd.DataFrame(parse_mallob("final/baseline/"))
dynamic_baseline_result = dynamic_baseline_df.result.rename("dynamic_baseline_result")

dynamic_baseline2_df = pd.DataFrame(parse_mallob("final/baseline2/"))
dynamic_baseline2_result = dynamic_baseline2_df.result.rename("dynamic_baseline2_result")

benchmark_dynamic_df = pd.DataFrame(parse_mallob("benchmark/dynamic/"))
benchmark_dynamic_result = benchmark_dynamic_df.result.rename("benchmark_dynamic_result")

# Row indexes should match since they are sorted in parse_mallob and written in correct order in the baseline.txt

all_result = pd.concat([ground_truth_result, dynamic_baseline_result, dynamic_baseline2_result, benchmark_dynamic_result], axis=1, join='inner')
all_result.groupby(['ground_truth_result', 'dynamic_baseline_result', "dynamic_baseline2_result", "benchmark_dynamic_result"]).size().reset_index().rename(columns={0:'count'})

In [35]:
gt = ground_truth_df.set_index("ID")["result"].rename("gt").str.upper()
bench = benchmark_dynamic_df.set_index("identifier")["result"].rename("bench")

result = pd.concat([gt, bench], axis=1, join="outer").fillna("UNKNOWN")

result.groupby(['gt', 'bench']).size().reset_index().rename(columns={0:'count'})

,gt,bench,count
0,SAT,SAT,126
1,SAT,UNKNOWN,58
2,UNKNOWN,SAT,1
3,UNKNOWN,UNKNOWN,72
4,UNSAT,UNKNOWN,61
5,UNSAT,UNSAT,82


In [20]:
ids = pd.read_csv('competition_scenario', delimiter=' ')
# ids.filename = ids.filename.astype(str)

results = pd.read_csv('competition_results', delimiter=' ')
# results.filename = results.filename.astype(str)

ground_truth_df = pd.merge(ids, results, on='filename')

test_df = pd.DataFrame(parse_mallob("benchmark/test/")) 

# For test
gt = ground_truth_df.set_index("ID")["result"].rename("gt").str.upper()
test = test_df.set_index("identifier")["result"].rename("test")

result = pd.concat([gt, test], axis=1, join="outer").fillna("UNKNOWN")

result.groupby(['gt', 'test']).size().reset_index().rename(columns={0:'count'})

,gt,test,count
0,SAT,SAT,12
1,SAT,UNKNOWN,172
2,UNKNOWN,UNKNOWN,73
3,UNSAT,UNKNOWN,130
4,UNSAT,UNSAT,13


In [37]:
pd.DataFrame(parse_generators("benchmark/dynamic/"))

Error: AttributeError